In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
from sys import stdout

## data

In [ ]:
from astropy.table import Table

In [ ]:
data = Table.read('http://svo2.cab.inta-csic.es/vocats/alhambra/download/alhambra.csv.gz', format='ascii.csv')

In [ ]:
data_good = (data['zb_1'] <= 1.0) & (data['Stellar_Flag'] <= 0.5) & (data['Satur_Flag'] == 0)

In [ ]:
data_z, data_M = data[data_good]['zb_1'], data[data_good]['M_ABS_1']

In [ ]:
len(data_z)

In [ ]:
bins_z = np.linspace(0, 1, 10)
bins_M = np.linspace(-24, -16, 20)

In [ ]:
data_zM, _, _ = np.histogram2d(data_z, data_M, bins=[bins_z, bins_M])

In [ ]:
plt.imshow(data_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

## model

In [ ]:
%cat alhambra.yml

In [ ]:
from skypy.pipeline import Pipeline

In [ ]:
model = Pipeline.read('alhambra.yml')

In [ ]:
model.execute()

In [ ]:
model_z = np.concatenate([model['SF.z'], model['Q.z']])
model_M = np.concatenate([model['SF.M'], model['Q.M']])

In [ ]:
model_zM, _, _ = np.histogram2d(model_z, model_M, bins=[bins_z, bins_M])

In [ ]:
plt.imshow(data_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

plt.imshow(model_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

In [ ]:
parameters = model.parameters.copy()

## inference

In [ ]:
from scipy.optimize import minimize

In [ ]:
def compute_covariance(parameters, n_samples=10):
    samples = []

    for i in range(n_samples):
        stdout.write('.')

        model.execute(parameters)

        model_z = np.concatenate([model['SF.z'], model['Q.z']])
        model_M = np.concatenate([model['SF.M'], model['Q.M']])

        model_zM, _, _ = np.histogram2d(model_z, model_M, bins=[bins_z, bins_M])

        samples.append(model_zM.reshape(-1))

    return np.linalg.pinv(np.cov(samples, rowvar=False), hermitian=True)

In [ ]:
def metric(values, W):
    model.execute(zip(parameters.keys(), values))

    model_z = np.concatenate([model['SF.z'], model['Q.z']])
    model_M = np.concatenate([model['SF.M'], model['Q.M']])

    model_zM, _, _ = np.histogram2d(model_z, model_M, bins=[bins_z, bins_M])

    x = data_zM.reshape(-1) - model_zM.reshape(-1)

    score = x@W@x

    if score < 0:
        score = np.inf

    stdout.write('.')

    return score

In [ ]:
def find_parameters(parameters, repeats):
    try:
        for i in range(repeats):
            stdout.write('covariance ')
            W = compute_covariance(parameters)
            stdout.write('\n')

            stdout.write('minimize ')
            result = minimize(metric, list(parameters.values()), args=(W,),
                              method='Nelder-Mead', options={'maxiter': 10})
            stdout.write('\n')

            parameters = dict(zip(parameters.keys(), result.x))
    except KeyboardInterrupt:
        stdout.write('\ninterrupted!')
    return parameters

In [ ]:
new_parameters = find_parameters(parameters, 3)

In [ ]:
model.execute(new_parameters)

In [ ]:
model_z = np.concatenate([model['SF.z'], model['Q.z']])
model_M = np.concatenate([model['SF.M'], model['Q.M']])

In [ ]:
model_zM, _, _ = np.histogram2d(model_z, model_M, bins=[bins_z, bins_M])

In [ ]:
np.sum(model_zM), np.sum(data_zM)

In [ ]:
plt.imshow(data_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

plt.imshow(model_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

In [ ]:
print(np.c_[list(parameters.values()),
            list(new_parameters.values())])